In [ ]:
#import all the necessary libraries  
import numpy as np
import random
import matplotlib.pyplot as plt
plt.style.use(['science','notebook', 'grid'])
import numba
from numba import njit
from scipy.ndimage import convolve, generate_binary_structure
from PIL import Image

In [ ]:
#create lattice size N, with randomly distributed up and down spins (half of the spins are pinted up und half down)
def lattice(N):
    lattice = np.zeros((N, N))
    random = np.random.random((N, N))
    lattice[random > 0.5 ] = 1
    lattice[random <= 0.5 ] = -1
    return lattice

#display lattice
def display_lattice(lattice):
    return Image.fromarray(np.uint8((lattice + 1) * 0.5 * 255))

In [ ]:
#metropolis algoritm for ising model
@numba.njit(nopython=True)
def MCS_lattice(lattice, N=100, BJ= 0.43): 
    lattice = lattice.copy()
    for b in range(N**2):
        i = random.randint(0,N-1)
        j = random.randint(0,N-1)
        initial = lattice[i][j]
        neigbours = lattice[i][(j-1)%N] + lattice[(i-1)%N][j] + lattice[i][(j+1)%N] + lattice[(i+1)%N][j]
        delta_E =  2 *  initial * neigbours
        if delta_E <= 0:
            lattice[i][j] = (-1) * initial 
        elif (delta_E > 0) * (random.random() < np.exp(- BJ * delta_E )): 
            lattice[i][j] = (-1) * initial     
    return(lattice)

In [ ]:
from ipywidgets import interact

def display_ising_sequence(images):
    def _show(frame=(0, len(images) - 1)):
        return display_lattice(images[frame])
    return interact(_show)


#amount of images to display set to 500
images = [lattice(100)]
for i in range(500):
    images.append(MCS_lattice(images[-1].copy()))
display_ising_sequence(images);

import numpy as np
from array2gif import write_gif
from __future__ import print_function
import numpy as np
from numba import jit
import time
from PIL import Image

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib.animation as animation
cmap1 = colors.ListedColormap(('#8e82fe','#580f41'))


def animate_run(images, size=(5,5), fps=15, bitrate=1800, filename='run', ticks='off', dpi=100, cmap=cmap1):

    """
        function used to produce run animations, takes awhile, suitable for small/short runs,
        has the advantage of producing clear images of any size
    """

    writer = animation.PillowWriter(fps=fps, metadata=dict(artist='Me'), bitrate=bitrate)
    FIG, ax = plt.subplots(figsize=size)
    ims = []
    plt.axis(ticks)
    
    for i in images:
        
        im = ax.imshow(i, origin='lower', cmap=cmap)
        ims.append([im])
        
    ani = animation.ArtistAnimation(FIG, ims, interval=150, blit=True, repeat_delay=1000)
    ani.save(filename + '.gif', dpi=dpi, writer=writer)

animate_run(images, filename='lattice_100x100_MCS')